In [1]:
import gensim.downloader as api
#model = api.load('word2vec-google-news-300')

In [1]:
from gensim.models import KeyedVectors


In [2]:
model2 = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300-SLIM.bin.gz', binary=True)

FileNotFoundError: [Errno 2] No such file or directory: 'GoogleNews-vectors-negative300-SLIM.bin.gz'

In [87]:
import pandas as pd
import nltk
from gensim.similarities import WmdSimilarity
import pickle
# from cassandra.cluster import Cluster
# from cassandra.query import BatchStatement
# from cassandra import ConsistencyLevel
# cluster = Cluster(['127.0.0.1'])#Cluster(['199.60.17.32','199.60.17.65'])
# session = cluster.connect('mcanute')

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/matt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [88]:
from nltk.corpus import stopwords
from nltk import download
stop_words = stopwords.words('english')
import gensim

In [89]:
from nltk import word_tokenize
nltk.download('punkt')  # Download data for tokenizer.

def preprocess(doc):
    doc = doc.lower()  # Lower the text.
    doc = word_tokenize(doc)  # Split into words.
    doc = [w for w in doc if not w in stop_words]  # Remove stopwords.
    doc = [w for w in doc if w.isalpha()]  # Remove numbers and punctuation.
    return doc

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\young\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [90]:
from pyemd import emd

In [8]:
data = pd.read_csv('websites/ParsedWebsitesCleanSample.csv')

In [101]:
# query = "SELECT * from Blogs;"
# data = pd.DataFrame(list(session.execute(query)))
data = pd.read_csv('ParsedBlogsCleanSamplev2.csv')

In [102]:
data.shape

(648, 10)

In [121]:
data = data.dropna() # !!!

In [122]:
data = data[data['body_text'].apply(len)>2] ## !!!

In [123]:
documents = data.values

In [124]:
documents.shape

(215, 10)

In [105]:
wmd_corpus = data['body_text'].apply(preprocess).values

In [21]:
instance = WmdSimilarity(wmd_corpus, model2, num_best=3)

In [22]:
pickle.dump(instance,open('WordMoverDistance-BlogInstanceSLIM.pkl','wb'))

In [106]:
instance = pickle.load(open('WordMoverDistance-BlogInstanceSLIM.pkl','rb'))

In [107]:
import re

In [108]:
def regex_text(val):
    if val:
        reg = re.compile(r'\b\w{3,20}\b')
        temp = reg.findall(val)
        temp_set = set(temp)
        temp_list = list(temp_set)
        if len(temp_list) < 20:
            temp2 = temp_list
        elif len(temp_list) < 50:
            temp2 = temp_list[10:]
        elif len(temp_list) < 100:
            temp2 = temp_list[30:]
        else:
            temp2 = temp_list[50:100]
        result = " ".join(temp2) 
        return ("..." + result + "....")
    else:
        return "Empty string found"

In [109]:
data.head(4)

,Unnamed: 0,website,back_color,body_font,body_text,last_update,link_color,text_color,title_font,title_text
0,0,https://lebleurenard.tumblr.com/,#4ad791,Helvetica Neue,Please send me asks ❤️ Moodboard for nights a...,2019-12-03 21:34:55,#A77DC2,#444444,Clarendon Text Pro,NaN
1,1,https://eboyanti.tumblr.com/,#FFD3EB,Helvetica Neue,i always laugh at how obnoxiously tall yukhei...,2019-12-04 05:44:36,#FFFFFF,#FFB8E3,Caslon FS,
2,2,https://trashout.tumblr.com/,#958383,Helvetica Neue,fe doobliedoos from twitter again then and no...,2019-11-02 14:19:26,#6AB3DF,#444444,Gibson,NaN
3,4,https://geminihurt.tumblr.com/,#CA2287,Helvetica Neue,Seeing their split reflection on a broken gla...,2019-12-01 22:00:32,#000000,#8d8eb6,Clearface FS,NaN


In [117]:
def similarityQuery(description):  # !!!
    description = description.replace('data', '')
    query = preprocess(description)
    sims = instance[query]  # A query is simply a "look-up" in the similarity class.

    # Print the query and the retrieved documents, together with their similarities.
    results = []
    for i in range(4):

        row = []
        row.append(documents[sims[i][0]][1]) # Website
        row.append(documents[sims[i][0]][2]) # Back_Color
        row.append(documents[sims[i][0]][6]) # Link_Color
        row.append(documents[sims[i][0]][7]) # Text_Color
        row.append(documents[sims[i][0]][8]) # Title_Font
        row.append(documents[sims[i][0]][3]) # Body_Font
        
        row.append(documents[sims[i][0]][4]) # Text
        
        row.append(sims[i][1])    # Similarity


        results.append(row)
    resultsDF = pd.DataFrame(results)

    resultsDF.columns = ['Website','BackColor','LinkColor','TextColor','TitleFont','BodyFont','Text','Similarity']
    #resultsDF['Fonts'] = resultsDF['Fonts'].replace({'\" \'': '\", \'', "\' \'": "\', \'", '(\r\n)': ','}, regex=True)
    #resultsDF['Colours'] = resultsDF['Colours'].replace({' ': ', '}, regex=True)
    resultsDF['Text'] = resultsDF['Text'].apply(regex_text)
    resultsDF = resultsDF.sort_values(by=['Similarity'],ascending=False)
    return resultsDF#.style.set_properties(subset=['Text'], **{'width': '350px'})

In [112]:
# similarityQuery('I will talk about some cool topics in programming')
# similarityQuery('i like anime')
similarityQuery('big data and machine learning')


big  and machine learning


,Website,BackColor,LinkColor,TextColor,TitleFont,BodyFont,Text,Similarity
0,https://masutrout.tumblr.com/,#FAFAFA,#529ECC,#444444,Gibson,Helvetica Neue,...machine softer....,0.530816
1,https://pencilshaevings.tumblr.com/,#ec5868,#fce7aa,#fce7aa,SimHei,Helvetica Neue,...MAGICAL cheeks hard have glasses bigger for big CHET work suizidale commission volume....,0.480309
2,https://hope-for-olicity.tumblr.com/,#847d81,#03131d,#444444,Georgia,Helvetica Neue,...should kill will you this Big Bad Your inability Barry realize Psst bite type the....,0.46983
3,https://ellanmwebb2.tumblr.com/,#FAFAFA,#529ECC,#444444,Gibson,Helvetica Neue,...Draw best Creating for here new some from prints Learning desk time November stay the Mega....,0.469686


In [119]:
import ast
def generateResult(description):
    df = similarityQuery(description)
    dict = {}
    aBC = [df.iloc[0]['BackColor'], df.iloc[1]['BackColor'], df.iloc[2]['BackColor'], df.iloc[3]['BackColor']]
    aLC = [df.iloc[0]['LinkColor'], df.iloc[1]['LinkColor'], df.iloc[2]['LinkColor'], df.iloc[3]['LinkColor']]
    aTC = [df.iloc[0]['TextColor'], df.iloc[1]['TextColor'], df.iloc[2]['TextColor'], df.iloc[3]['TextColor']]
    aTF = [df.iloc[0]['TitleFont'], df.iloc[1]['TitleFont'], df.iloc[2]['TitleFont'], df.iloc[3]['TitleFont']]
    aBF = [df.iloc[0]['BodyFont'], df.iloc[1]['BodyFont'], df.iloc[2]['BodyFont'], df.iloc[3]['BodyFont']]
    
    dict['back_color'] = aBC
    dict['link_color'] = aLC
    dict['text_color'] = aTC
    dict['title_font'] = aTF
    dict['body_font'] = aBF
    
    return dict

In [125]:
generateResult("i will talk about cool tricks in machine learning and big data")

IndexError: index 316 is out of bounds for axis 0 with size 215